In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (9,6)
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

Using all features except containing data leakage.

In [44]:
model_features = [
    # Borrower info
    'annual_inc', 'verification_status', 'emp_length', 'home_ownership', 'addr_state',
    'dti', 'purpose', 'application_type', 'emp_title', 'desc',

    # Credit risk indicators
    'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq',
    'mths_since_last_record', 'open_acc', 'total_acc', 'pub_rec', 'revol_bal',
    'revol_util', 'collections_12_mths_ex_med', 'mths_since_last_major_derog',
    'delinq_2yrs', 'acc_now_delinq', 'num_tl_90g_dpd_24m', 'policy_code',
    'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
    'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
    'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
    'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal',
    'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt',
    'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op',
    'mo_sin_rcnt_tl', 'earliest_cr_line', 'mort_acc', 'mths_since_recent_bc',
    'mths_since_recent_bc_dlq', 'mths_since_recent_inq',
    'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl',
    'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
    'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m',
    'num_tl_30dpd', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
    'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim',
    'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit','issue_d',

    # Loan application terms
    'loan_amnt', 'int_rate', 'term', 'installment', 'grade', 'sub_grade',

    # joint and secondary applicant info
    'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'revol_bal_joint',
    'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line',
    'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc',
    'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts',
    'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med',
    'sec_app_mths_since_last_major_derog',

    # status
    'loan_status'
]

In [45]:
model_df = pd.read_csv("/content/drive/My Drive/lending_club_data/accepted_2007_to_2018Q4.csv",low_memory=False,usecols=model_features)

### Feature Engineering

Credit Utilization and Ratios.

In [46]:
model_df['credit_card_util_pct'] = model_df['revol_bal'] / (model_df['total_rev_hi_lim'] + 1)
model_df['installment_income_ratio'] = model_df['installment'] / (model_df['annual_inc'] + 1)
model_df['loan_amount_income_ratio'] = model_df['loan_amnt'] / (model_df['annual_inc'] + 1)

Delinquency & Derogatory Flags

In [47]:
model_df['had_recent_major_derog'] = (model_df['mths_since_last_major_derog'] < 12).astype(int)
model_df['delinq_flag'] = (model_df['mths_since_last_delinq'] < 12).astype(int)
model_df['bankruptcy_flag'] = (model_df['pub_rec_bankruptcies'] > 0).astype(int)
model_df['historical_delinquency_rate'] = model_df['delinq_2yrs'] / (model_df['total_acc'] + 1)

Behavioral Indicators

In [48]:
model_df['recent_open_accs'] = (
    model_df[['open_acc_6m', 'open_il_12m', 'open_il_24m', 'open_rv_12m', 'open_rv_24m']]
    .sum(axis=1, skipna=True)
)

In [49]:
model_df['recent_inquiries_sum'] = (
    model_df[['inq_last_6mths', 'inq_fi', 'inq_last_12m', 'sec_app_inq_last_6mths']]
    .sum(axis=1, skipna=True)
)

Interaction Features

In [50]:
model_df['is_major_derog_and_util_high'] = (
    ((model_df['mths_since_last_major_derog'] < 12) & (model_df['revol_util'] > 80))
).astype(int)

In [51]:
model_df['desc_contains_debt'] = model_df['desc'].fillna('').str.contains(r'debt', case=False).astype(int)

### Cleaning

In [52]:
keep_statuses = [
    'Fully Paid', 'Charged Off', 'Default',
    'Does not meet the credit policy. Status: Fully Paid',
    'Does not meet the credit policy. Status: Charged Off'
]

model_df = model_df[model_df['loan_status'].isin(keep_statuses)]

default_statuses = ['Charged Off', 'Default', 'Does not meet the credit policy. Status: Charged Off']

model_df['loan_status'] = model_df['loan_status'].apply(lambda x: 1 if x in default_statuses else 0)

In [53]:
model_df['home_ownership'] = model_df['home_ownership'].replace({'ANY': 'OTHER', 'NONE': 'OTHER', 'OTHER': 'OTHER'})

### Encoding and transformations

In [54]:
one_hot_cols = ['verification_status','home_ownership','purpose']
model_df = pd.get_dummies(model_df,columns=one_hot_cols,drop_first=True)

In [55]:
emp_map = {
    '< 1 year': 0,
    '1 year': 1, '2 years': 2, '3 years': 3, '4 years': 4,
    '5 years': 5, '6 years': 6, '7 years': 7, '8 years': 8,
    '9 years': 9, '10+ years': 10
}
model_df['emp_length'] = model_df['emp_length'].map(emp_map)

In [56]:
model_df['application_type'] = model_df['application_type'].map({'Individual' : 0,'Joint App':1})
model_df['term'] = model_df['term'].map({' 36 months' : 0, ' 60 months' : 1})

In [57]:
model_df = model_df[model_df['dti']>0]

In [58]:
addr_freq = model_df['addr_state'].value_counts(normalize=True)
model_df['addr_state_freq'] = model_df['addr_state'].map(addr_freq)
model_df = model_df.drop(columns=['addr_state'])

/tmp/ipython-input-58-731542170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['addr_state_freq'] = model_df['addr_state'].map(addr_freq)


In [59]:
subgrade_order = []
for g_idx, g in enumerate(['A', 'B', 'C', 'D', 'E', 'F', 'G'], 1):
    for s in range(1, 6):
        subgrade_order.append(f"{g}{s}")

subgrade_float_map = {sg: float(f"{g_idx}.{s}") for sg, (g_idx, s) in zip(subgrade_order, [(i//5 + 1, i%5 + 1) for i in range(35)])}

model_df['sub_grade'] = model_df['sub_grade'].map(subgrade_float_map)



In [60]:
model_df = model_df.drop(columns=['grade'])

In [61]:
(model_df['revol_util']>100).value_counts()

,count
revol_util,
False,1339430
True,4685


Values over 100% are often invalid, noisy, or result from data entry errors so, we gonna clip them.

In [62]:
model_df['revol_util'] = model_df['revol_util'].clip(upper=100)

In [63]:
(model_df['fico_range_high'] - model_df['fico_range_low']).value_counts()


,count
4.0,1343930
5.0,185


In [64]:
model_df['fico_score'] = (model_df['fico_range_low'] + model_df['fico_range_high']) / 2
model_df.drop(['fico_range_low', 'fico_range_high'], axis=1, inplace=True)

Fico too DTI ratio

In [65]:
model_df['fico_dti_ratio'] = model_df['fico_score'] / (1 + model_df['dti'])

log transform skewed features.

In [66]:
model_df['annual_inc_log'] = np.log1p(model_df['annual_inc'])
model_df = model_df.drop(columns=['annual_inc'])

In [67]:
model_df['dti_log'] = np.log1p(model_df['dti'])
model_df = model_df.drop(columns=['dti'])

**collections_12_mths_ex_med** Skewed and peaky, but many zero. bining will be better option.

In [68]:
model_df['collections_12_mths_ex_med'].isnull().sum()

np.int64(54)

In [69]:
model_df['collections_12_mths_ex_med'] = model_df['collections_12_mths_ex_med'].fillna(0)

In [70]:
model_df['collections_bins'] = pd.cut(model_df['collections_12_mths_ex_med'],
                                      bins=[-1,0,1,np.inf],
                                      labels=[0,1,2]).astype(int)
model_df.drop(columns=['collections_12_mths_ex_med'], inplace=True)

The difference is not much so, we'll use just average.

In [71]:
model_df['revol_bal_log'] = np.log1p(model_df['revol_bal'])
model_df = model_df.drop(columns=['revol_bal'])

In [72]:
model_df['tot_coll_amt'].skew()

np.float64(670.3283453309389)

In [73]:
model_df['tot_coll_amt_log'] = np.log1p(model_df['tot_coll_amt'])
model_df = model_df.drop(columns=['tot_coll_amt'])

In [74]:
model_df['tot_cur_bal'].skew()

np.float64(2.9012336564938908)

In [75]:
model_df['tot_cur_bal_log'] = np.log1p(model_df['tot_cur_bal'])
model_df = model_df.drop(columns=['tot_cur_bal'])

In [76]:
from datetime import datetime

model_df['earliest_cr_line'] = pd.to_datetime(model_df['earliest_cr_line'],format='%b-%Y', errors='coerce')
model_df['issue_d'] = pd.to_datetime(model_df['issue_d'],format='%b-%Y', errors='coerce')

model_df['account_age_days'] = (model_df['issue_d'] - model_df['earliest_cr_line']).dt.days
model_df['account_age_years'] = model_df['account_age_days'] / 365.25


In [77]:
model_df['sec_app_earliest_cr_line'] = pd.to_datetime(model_df['sec_app_earliest_cr_line'], format='%b-%Y', errors='coerce')


model_df['sec_app_credit_age_days'] = (model_df['issue_d'] - model_df['sec_app_earliest_cr_line']).dt.days
model_df['sec_app_credit_age_years'] = model_df['sec_app_credit_age_days'] / 365.25


In [78]:
model_df.drop(['earliest_cr_line', 'issue_d','account_age_days','sec_app_earliest_cr_line','sec_app_credit_age_days'], axis=1, inplace=True)

In [79]:
model_df['annual_inc_joint_log'] = np.log1p(model_df['annual_inc_joint'])
model_df['dti_joint_log'] = np.log1p(model_df['dti_joint'])

model_df = model_df.drop(columns=['annual_inc_joint','dti_joint'])

In [80]:
model_df = pd.get_dummies(model_df,columns=['verification_status_joint'],dummy_na=True)

In [81]:
model_df['revol_bal_joint_log'] = np.log1p(model_df['revol_bal_joint'])
model_df = model_df.drop(columns=['revol_bal_joint'])

In [82]:
model_df['sec_app_fico_score'] = (model_df['sec_app_fico_range_low'] + model_df['sec_app_fico_range_high']) / 2
model_df.drop(['sec_app_fico_range_low', 'sec_app_fico_range_high'], axis=1, inplace=True)

In [83]:
model_df['sec_app_revol_util'] = model_df['sec_app_revol_util'].clip(upper=100)

In [84]:
model_df['sec_app_collections_bins'] = pd.cut(
    model_df['sec_app_collections_12_mths_ex_med'],
    bins=[-1, 0, 1, np.inf],
    labels=[0, 1, 2]
)

# convert labels to integers, preserving NaN
model_df['sec_app_collections_bins'] = model_df['sec_app_collections_bins'].astype('float')

model_df.drop(columns=['sec_app_collections_12_mths_ex_med'], inplace=True)


In [85]:
emp_title_counts = model_df['emp_title'].value_counts()
model_df['emp_title_freq'] = model_df['emp_title'].map(emp_title_counts)

In [86]:
model_df.drop(columns=['emp_title'], inplace=True)

In [87]:
model_df['has_desc'] = model_df['desc'].notnull().astype(int)

default_rate_by_desc = model_df.groupby('has_desc')['loan_status'].agg(['mean', 'count'])
default_rate_by_desc.rename(columns={'mean': 'default_rate', 'count': 'num_loans'}, inplace=True)
print(default_rate_by_desc)

          default_rate  num_loans
has_desc                         
0             0.204342    1220756
1             0.153268     123359




*   Only 9% of the applications have description.
*   Borrowers who wrote a description defaulted less by ~5 percentage points.



In [88]:
model_df.drop(columns=['desc'], inplace=True)

### More features

Flaging missing value features that are important.

In [89]:
model_df['inq_last_6mths_missing'] = model_df['inq_last_6mths'].isna().astype(int)
model_df['mths_since_last_delinq_missing'] = model_df['mths_since_last_delinq'].isna().astype(int)
model_df['mths_since_last_record_missing'] = model_df['mths_since_last_record'].isna().astype(int)
model_df['revol_util_missing'] = model_df['revol_util'].isna().astype(int)
model_df['mths_since_last_major_derog_missing'] = model_df['mths_since_last_major_derog'].isna().astype(int)
model_df['num_tl_90g_dpd_24m_missing'] = model_df['num_tl_90g_dpd_24m'].isna().astype(int)

### Store cleaned data Frame

In [91]:
model_df.to_parquet('/content/drive/MyDrive/model_df.parquet')